In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the data from the CSV file
df = pd.read_csv('cleaned_adw_df_22.csv')
cols_to_remove = ['Delay National Aviation System (Minutes)_a_e', 'Delay Security (Minutes)_a_e', 'Delay Late Aircraft Arrival (Minutes)_a_e', 
 'Delay National Aviation System (Minutes)_d_e', 'Delay Security (Minutes)_d_e', 'Delay Late Aircraft Arrival (Minutes)_d_e', 
 'Delay National Aviation System (Minutes)_a_l', 'Delay Security (Minutes)_a_l', 'Delay Late Aircraft Arrival (Minutes)_a_l', 
 'Delay National Aviation System (Minutes)_d_l', 'Delay Security (Minutes)_d_l', 'Delay Late Aircraft Arrival (Minutes)_d_l','Date (MM/DD/YYYY)_d_e']
df =df.drop(columns=cols_to_remove)
# filtered data based on year
years_to_filter = {2019,2020,2021,2022,2023}
df = df[df['Year_e'].isin(years_to_filter)]

airport = 'MCO'

df = df[df['Origin Airport_a']==airport]

df.head(3)

Origin Airport_a  Scheduled Elapsed Time (Minutes)_a_e  \
705              MCO                                 166.0   
706              MCO                                 166.0   
707              MCO                                 166.0   

     Arrival Delay (Minutes)_a_e  temp_a_e wx_phrase_a_e  dewPt_a_e  \
705                         -8.0      32.0        Cloudy       22.0   
706                         47.0      32.0    Clear/Calm       15.0   
707                        -14.0      64.0         Rainy       46.0   

     heat_index_a_e  rh_a_e  pressure_a_e  vis_a_e  ...  snow_hrly_d_l  \
705            32.0    66.0         29.62     10.0  ...            0.0   
706            32.0    50.0         29.68     10.0  ...            0.0   
707            64.0    52.0         29.26     10.0  ...            0.0   

     uv_desc_d_l  feels_like_d_l  uv_index_d_l clds_d_l  Main_Carrier_l  \
705          Low            66.0           0.0      OVC              B6   
706          Low            69.0           0.0      FEW              B6   
707          Low            72.0           0.0      FEW              B6   

     Week_Category_l A_Time_of_Day_l  Year_l time_diff  
705          Weekend           Night    2019      4.93  
706          Weekend           Night    2019      4.93  
707          Weekend           Night    2019      4.93  

[3 rows x 74 columns]

In [7]:
# Convert categorical variables to numerical using one-hot encoding
categorical_cols = ['Origin Airport_a', 'wx_phrase_a_e', 'uv_desc_a_e', 'clds_a_e', 'wx_phrase_d_e', 'uv_desc_d_e', 
                    'clds_d_e', 'Main_Carrier_e', 'Week_Category_e', 'A_Time_of_Day_e', 'wx_phrase_a_l', 'uv_desc_a_l',
                    'clds_a_l', 'wx_phrase_d_l', 'uv_desc_d_l', 'clds_d_l', 'Main_Carrier_l', 'Week_Category_l', 'A_Time_of_Day_l']
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Select only numeric columns
numeric_cols = df.select_dtypes(include=['number'])

# Calculate correlation with "Arrival Delay (Minutes)_a" and sort by absolute value
corr_with_delay = df.corr()['Arrival Delay (Minutes)_a_l'].abs().sort_values(ascending=False)

# print(corr_with_delay)

# Select top 12 features with low intercorrelation
selected_features = []
for feature in corr_with_delay.index:
    if len(selected_features) >= 30:
        break
    if corr_with_delay[feature] >= -0.1:
        selected_features.append(feature)
        
selected_features.remove('Arrival Delay (Minutes)_a_l')
print(selected_features)

# Print remaining numerical columns
remaining_numeric_cols = [col for col in numeric_cols.columns if col not in selected_features]
remaining_numeric_cols.remove('Arrival Delay (Minutes)_a_l')
print("Remaining Numerical Columns:")
print(remaining_numeric_cols)

# Remove the specified columns

columns_to_remove = remaining_numeric_cols
# .append('Year')

df = df.drop(columns=remaining_numeric_cols)

['precip_hrly_d_e', 'uv_desc_a_e_Moderate', 'Main_Carrier_l_WN', 'uv_desc_a_e_Low', 'Arrival Delay (Minutes)_a_e', 'wx_phrase_a_l_Cloudy', 'uv_index_a_e', 'wx_phrase_d_e_Rainy', 'uv_desc_d_e_Low', 'clds_a_l_FEW', 'uv_desc_d_e_Moderate', 'Scheduled Elapsed Time (Minutes)_a_l', 'wx_phrase_d_l_Stormy', 'wspd_a_l', 'uv_index_d_e', 'clds_d_e_CLR', 'clds_d_l_OVC', 'precip_hrly_a_e', 'vis_a_e', 'precip_hrly_a_l', 'uv_index_d_l', 'A_Time_of_Day_e_Morning Peak', 'clds_d_e_OVC', 'uv_desc_d_l_Low', 'precip_total_d_e', 'uv_desc_a_l_Moderate', 'uv_desc_a_l_Low', 'Main_Carrier_e_WN', 'Week_Category_e_StartOfWeek']
Remaining Numerical Columns:
['Scheduled Elapsed Time (Minutes)_a_e', 'temp_a_e', 'dewPt_a_e', 'heat_index_a_e', 'rh_a_e', 'pressure_a_e', 'wspd_a_e', 'precip_total_a_e', 'snow_hrly_a_e', 'feels_like_a_e', 'temp_d_e', 'dewPt_d_e', 'heat_index_d_e', 'rh_d_e', 'pressure_d_e', 'vis_d_e', 'wspd_d_e', 'snow_hrly_d_e', 'feels_like_d_e', 'Year_e', 'temp_a_l', 'dewPt_a_l', 'heat_index_a_l', 'rh_a_

In [8]:
# Define the target variable based on arrival delay
df['Class of Arrival Time'] = pd.cut(df['Arrival Delay (Minutes)_a_l'],
                                       bins=[float('-inf'), -5, 5, float('inf')],
                                       labels=['Early', 'On-time', 'Late'])

# Drop unnecessary columns and the target variable
X = df.drop(['Arrival Delay (Minutes)_a_l', 'Class of Arrival Time'], axis=1)
y = df['Class of Arrival Time']
# y = df['Arrival Delay (Minutes)_a']


# Convert categorical variables to numerical using one-hot encoding
# categorical_cols = ['Origin Airport_a', 'wx_phrase_a', 'uv_desc_a', 'clds_a',
#                     'wx_phrase_d', 'uv_desc_d', 'clds_d', 'Main_Carrier', 'Week_Category', 'A_Time_of_Day']
# X_encoded = pd.get_dummies(X, columns=categorical_cols, drop_first=True)
X_encoded = X

# Standardize numerical columns
numerical_cols = selected_features
scaler = StandardScaler()
X_encoded[numerical_cols] = scaler.fit_transform(X_encoded[numerical_cols])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y,stratify = y, test_size=0.2, random_state=42)

# scaler = StandardScaler()
# X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
# X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import mean_absolute_error
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score

# Identify the model with the highest accuracy
best_model_name = None
best_accuracy = 0


# Initialize and train different models
models = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'SVM': SVC(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42,n_estimators = 250,max_depth = 50, learning_rate= 1 ),
    'ada' : AdaBoostClassifier(random_state=42),
# 'xg':  XGBClassifier(random_state=42),
'knn' : KNeighborsClassifier(n_neighbors=15),
'dtree':  DecisionTreeClassifier(max_depth=7,random_state=42),
'rand_class':  RandomForestClassifier(max_depth=15,random_state=42),
'GNB':  GaussianNB()
}

for model_name, model in models.items():

    model.fit(X_train, y_train)
    if model_name =='Logistic Regression':
        print(model.coef_,model.intercept_)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'{model_name} Accuracy: {accuracy:.2f}')
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model_name = model_name
    # mae = mean_absolute_error(y_test, y_pred)
    # mae_ratio = mae / np.mean(y_test)
    # print("MAE:", mae)
    # print("MAE Ratio:", mae_ratio)
    # print(model.score(X_test,y_test))
    # print(classification_report(y_test, y_pred))
    # print('-' * 50)


# Select the best model
best_model = models[best_model_name]

print("model with most accuracy is " + str(best_model))

[[-7.19017869e-02  4.56213344e-01  3.26156628e-01  2.10022268e-01
   2.97899186e-01 -4.15566384e-01  3.54394891e-01  3.34004181e-02
  -2.51562748e-01 -2.39200111e-01 -1.69232976e-02 -1.40740569e-01
   1.59199292e-01 -6.37992880e-01  3.17751996e-01  1.13122849e-01
   4.40310081e-01 -5.42417740e-03  2.63175953e-01 -9.32809268e-01
  -5.00667658e-01  3.84006455e-01  3.05319869e-01 -2.14112291e-01
  -2.74560065e-02 -4.94873696e-02  6.70885386e-01  4.22531173e-01
   1.64343148e-02  1.11671425e-01 -5.10176389e-02  1.77211424e-01
   1.70493801e-01 -6.89560390e-02  2.10080241e-02 -9.37748230e-05
   2.74282639e-01 -5.22769097e-01  1.52407058e-01 -8.24614064e-02
   4.05211330e-01  3.43756585e-01 -4.98364735e-01 -3.09099956e-02
   3.51879421e-01 -1.61801991e-01  2.45967709e-02 -3.29747315e-01
   5.70323951e-02  6.14727013e-01 -5.69421652e-01  2.12006401e-01
  -1.98453162e-01  5.76160447e-01 -4.43473407e-01 -4.37808804e-01
  -8.54177221e-02 -9.36803108e-03 -9.22773858e-02  3.95548694e-01
  -2.72311

/opt/homebrew/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Gradient Boosting Accuracy: 0.45
ada Accuracy: 0.48
knn Accuracy: 0.47
dtree Accuracy: 0.46
rand_class Accuracy: 0.48
GNB Accuracy: 0.34
model with most accuracy is RandomForestClassifier(random_state=42)


In [11]:

# Load the data from the CSV file
df = pd.read_csv('CIS_662_FINAL_Predictions.csv')

# Filter rows with ORIGIN as JFK
airport_data_index = df[df['Origin Airport_a'] == 'JFK'].index

# Change ARRIVAL STATUS_Prev_flight to 'early' and make predictions
df.loc[airport_data_index, 'ARRIVAL STATUS_Prev_flight'] = 'early'

# Perform one-hot encoding on the JFK data using previous encoder
airport_data_encoded_early = pd.get_dummies(df.loc[airport_data_index], columns=categorical_cols, drop_first=True)

# Standardize numerical columns using previous scaler
airport_data_encoded_early[numerical_cols] = scaler.transform(airport_data_encoded_early[numerical_cols])

# Make predictions using the selected model
predictions_early = best_model.predict(airport_data_encoded_early[X_train.columns])

df.loc[airport_data_index, 'ARRIVAL STATUS_Prev_flight_early'] = predictions_early

print(airport + " airport prediction for ARRIVAL STATUS_Prev_flight_early are created")


# Change ARRIVAL STATUS_Prev_flight to 'ontime' and make predictions
df.loc[airport_data_index, 'ARRIVAL STATUS_Prev_flight'] = 'ontime'

# Perform one-hot encoding on the JFK data using previous encoder
airport_data_encoded_ontime = pd.get_dummies(df.loc[airport_data_index], columns=categorical_cols, drop_first=True)

# Standardize numerical columns using previous scaler
airport_data_encoded_ontime[numerical_cols] = scaler.transform(airport_data_encoded_ontime[numerical_cols])

# Make predictions using the selected model
predictions_ontime = best_model.predict(airport_data_encoded_ontime[X_train.columns])


df.loc[airport_data_index, 'ARRIVAL STATUS_Prev_flight_ontime'] = predictions_ontime

print(airport + " airport prediction for ARRIVAL STATUS_Prev_flight_ontime are created")



# Change ARRIVAL STATUS_Prev_flight to 'late' and make predictions
df.loc[airport_data_index, 'ARRIVAL STATUS_Prev_flight'] = 'late'

# Perform one-hot encoding on the JFK data using previous encoder
airport_data_encoded_late = pd.get_dummies(df.loc[airport_data_index], columns=categorical_cols, drop_first=True)

# Standardize numerical columns using previous scaler
airport_data_encoded_late[numerical_cols] = scaler.transform(airport_data_encoded_late[numerical_cols])

# Make predictions using the selected model
predictions_late = best_model.predict(airport_data_encoded_late[X_train.columns])

df.loc[airport_data_index, 'ARRIVAL STATUS_Prev_flight_late'] = predictions_late

print(airport + " airport prediction for ARRIVAL STATUS_Prev_flight_late are created")


# Save the updated DataFrame back to the same file
df.to_csv('CIS_662_FINAL_Predictions.csv', index=False)


MCO airport prediction for ARRIVAL STATUS_Prev_flight_early are created
MCO airport prediction for ARRIVAL STATUS_Prev_flight_ontime are created
MCO airport prediction for ARRIVAL STATUS_Prev_flight_late are created
